# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems.

In [ ]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("Linear Regression").getOrCreate()

In [ ]:
# check (try) if Spark session variable (spark) exists and print information about the Spark context
try:
    spark
except NameError:
    print("Spark session does not context exist. Please create Spark session first (run cell above).")
else:
    configurations = spark.sparkContext.getConf().getAll()
    for item in configurations: print(item)

In [ ]:
#import numpy as np
from pyspark.ml.regression import LinearRegression

In [ ]:
# Load training data
training = spark.read.format("libsvm").load("data/sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [ ]:
training.show()

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [ ]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [ ]:
# Fit the model
lrModel = lr.fit(training)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

There is a summary attribute that contains even more info!

In [ ]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [ ]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [ ]:
all_data = spark.read.format("libsvm").load("data/sample_linear_regression_data.txt")

In [ ]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [ ]:
train_data.show()

In [ ]:
test_data.show()

In [ ]:
unlabeled_data = test_data.select('features')

In [ ]:
unlabeled_data.show()

Now we only train on the train_data

In [ ]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [ ]:
test_results = correct_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [ ]:
predictions = correct_model.transform(unlabeled_data)

In [ ]:
predictions.show()

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!

# Linear Regression Code Along

Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [ ]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("data/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [ ]:
# Print the Schema of the DataFrame
data.printSchema()

In [ ]:
data.show()

In [ ]:
data.head()

In [ ]:
for item in data.head():
    print(item)

## Setting Up DataFrame for Machine Learning 

In [ ]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

In [ ]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [ ]:
output = assembler.transform(data)

In [ ]:
output.select("features").show()

In [ ]:
output.show()

In [ ]:
final_data = output.select("features",'Yearly Amount Spent')

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.describe().show()

In [ ]:
test_data.describe().show()

In [ ]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
# Interesting results....
test_results.residuals.show()

In [ ]:
unlabeled_data = test_data.select('features')

In [ ]:
predictions = lrModel.transform(unlabeled_data)

In [ ]:
predictions.show()

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

Excellent results! Let's see how you handle some more realistically modeled data in the Consulting Project!

# Data Transformations

You won't always get data in a convienent format, often you will have to deal with data that is non-numerical, such as customer names, or zipcodes, country names, etc...

A big part of working with data is using your own domain knowledge to build an intuition of how to deal with the data, sometimes the best course of action is to drop the data, other times feature-engineering is a good way to go, or you could try to transform the data into something the Machine Learning Algorithms will understand.

Spark has several built in methods of dealing with thse transformations, check them all out here: http://spark.apache.org/docs/latest/ml-features.html

Let's see some examples of all of this!

In [ ]:
df = spark.read.csv('data/fake_customers.csv',inferSchema=True,header=True)

In [ ]:
df.show()

## Data Features

### StringIndexer

We often have to convert string information into numerical information as a categorical feature. This is easily done with the StringIndexer Method:

In [ ]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["user_id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

The next step would be to encode these categories into "dummy" variables.

### VectorIndexer

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order. 

Assume that we have a DataFrame with the columns id, hour, mobile, userFeatures, and clicked:

     id | hour | mobile | userFeatures     | clicked
    ----|------|--------|------------------|---------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0
     
userFeatures is a vector column that contains three user features. We want to combine hour, mobile, and userFeatures into a single feature vector called features and use it to predict clicked or not. If we set VectorAssembler’s input columns to hour, mobile, and userFeatures and output column to features, after transformation we should get the following DataFrame:

     id | hour | mobile | userFeatures     | clicked | features
    ----|------|--------|------------------|---------|-----------------------------
     0  | 18   | 1.0    | [0.0, 10.0, 0.5] | 1.0     | [18.0, 1.0, 0.0, 10.0, 0.5]

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])
dataset.show()

In [ ]:
assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show()

There ar emany more data transformations available, we will cover them once we encounter a need for them, for now these were the most important ones.

Let's continue on to Linear Regression!

# Linear Regression Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 


In [ ]:
df = spark.read.csv('data/cruise_ship_info.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()

In [ ]:
df.show()

In [ ]:
df.describe().show()

## Dealing with the Cruise_line categorical variable
Ship Name is a useless arbitrary string, but the cruise_line itself may be useful. Let's make it into a categorical variable!

In [ ]:
df.groupBy('Cruise_line').count().show()

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)
indexed.head(5)

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
indexed.columns

In [ ]:
assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [ ]:
output = assembler.transform(indexed)

In [ ]:
output.select("features", "crew").show()

In [ ]:
final_data = output.select("features", "crew")

In [ ]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [ ]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [ ]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

In [ ]:
# R2 of 0.86 is pretty good, let's check the data a little closer
from pyspark.sql.functions import corr

In [ ]:
df.select(corr('crew','passengers')).show()

In [ ]:
df.select(corr('crew','cabins')).show()

Okay, so maybe it does make sense! Well that is good news for us, this is information we can bring to the company!


## Stop The Spark Session

In [ ]:
# stop the underlying SparkContext.
try:
    spark
except NameError:
    print("Spark session does not context exist - nothing to stop.")
else:
    spark.stop()